In [1]:
from langchain_ollama import OllamaLLM
import evaluate
from datasets import load_dataset

c:\Users\sathv\anaconda3\envs\ragenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = OllamaLLM(model="hf.co/sathvik123/llama3-finnace-chat-gguf")

In [3]:
llm.invoke("Is it wise to invest small amounts of money short-term?")

"It's generally considered wise to invest your money in a responsible and informed manner. However, investing small amounts of money short-term can be a good strategy for several reasons:\n\n1. Compound interest: Investing small amounts regularly over time allows you to take advantage of compound interest. This means that the interest earned on your investment will earn interest itself, leading to exponential growth.\n\n2. Diversification: Investing small amounts in different assets or securities helps to diversify your portfolio, reducing risk and increasing potential returns.\n\n3. Habit formation: Investing small amounts regularly can help you develop a savings habit and build discipline in managing your finances.\n\n4. Low-cost investing: With the advent of low-cost index funds or ETFs, it's possible to invest with lower fees compared to actively managed funds. This can be especially beneficial for short-term investments where frequent buying and selling may occur.\n\n5. Emotional 

In [4]:
def load_data():
    dataset = load_dataset("poornima9348/finance-alpaca-1k-test", split="test")
    return dataset[:100]

In [5]:
def generate_predictions(llm, dataset, prompt_template):
    predictions = []
    references = []

    for row in dataset["instruction"]:
        input = prompt_template.format(row)
        response = llm.invoke(input)
        predictions.append(response.strip())

    for row in dataset["output"]:
        references.append(row.strip())

    return predictions, references

In [6]:
def compute_scores(predictions, references):
    rouge_metric = evaluate.load("rouge")
    meteor_metric = evaluate.load("meteor")

    rouge_score = rouge_metric.compute(predictions=predictions, references=references)
    meteor_score = meteor_metric.compute(predictions=predictions, references=references)

    return {
        "ROUGE": rouge_score,
        "METEOR": meteor_score
    }

In [7]:
my_prompt = """
### Instruction:
You are a financial advisor chatbot that provides clear, accurate, and helpful responses to financial questions. Explain financial concepts in simple terms, offer relevant examples when appropriate, and ensure your answers are concise, objective, and tailored to the user's query.

### Input:
{}
"""

In [8]:
data = load_data()

In [9]:
# Generate predictions
predictions, references = generate_predictions(llm, data, my_prompt)

In [10]:
compute_scores(predictions, references)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sathv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\sathv\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sathv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


{'ROUGE': {'rouge1': 0.22327770766125699,
  'rouge2': 0.03247719263014715,
  'rougeL': 0.10862682050052778,
  'rougeLsum': 0.14947534225092715},
 'METEOR': {'meteor': 0.224027052286671}}